In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
tnews_ds = load_dataset("clue/clue", "tnews")

In [ ]:
tnews_ds

In [ ]:
tnews_ds["train"][0]

In [ ]:
labels = train_set.features["label"]
labels

In [ ]:
type(labels)

In [ ]:
!wget https://storage.googleapis.com/cluebenchmark/tasks/tnews_public.zip

In [ ]:
!ls 

In [ ]:
!unzip tnews_public.zip

In [ ]:
!ls

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines

labels = []
with jsonlines.open("labels.json") as reader:
    for obj in reader:
        labels.append(obj)

In [ ]:
from pprint import pprint
pprint(labels)
print(len(labels))

## 文本分类

### 1 分词预处理

In [ ]:
tnews_ds

In [ ]:
valid_set = tnews_ds["validation"]
valid_set[0]

In [ ]:
label_names = valid_set.features["label"].names
label_names

In [ ]:
labels

In [ ]:
label_descs = {}
for label in labels:
    k, v = label["label"], label["label_desc"]
    label_descs[k] = v

label_descs

In [ ]:
id2labels = {i: v for i, v in enumerate(label_names)}
id2labels

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("TsinghuaAI/CPM-Generate")

In [ ]:
def preprocess(samples):
    tokenized = tokenizer(
        samples["sentence"],
        truncation=True,
        padding="max_length",
        # max_length=model.config.n_ctx
        max_length=1024
    )
    tokenized["label_desc"] = [id2labels[l] for l in samples["label"]]
    return tokenized

# 应用预处理
dataset = valid_set.map(preprocess, batched=True)

In [ ]:
dataset

In [ ]:
dataset[103]["label_desc"]

### 2 加载模型

In [ ]:
from transformers import AutoModelForSequenceClassification

ckpt = "TsinghuaAI/CPM-Generate"
model = AutoModelForSequenceClassification.from_pretrained(
    ckpt,
    num_labels=len(label_names),
    id2label=id2labels
)

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline(
    "text-classification", 
    # model="TsinghuaAI/CPM-Generate",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
sents = [dataset[i]["sentence"] for i in range(10)]
sents

In [ ]:
results = classifier(sents)
results

## 使用准确率作为分类评测指标

In [ ]:
!pip install --quiet evaluate

In [ ]:
import evaluate

In [45]:
acc_computer = evaluate.load("accuracy")

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [46]:
# count = 0
for sample in tqdm(dataset, desc="Classifing"):
    result = classifier(sample["sentence"])
    expected = sample["label_desc"]
    actual = result[0]["label"]
    # print(expected, actual)
    acc_computer.add(references=expected, predictions=actual)
    # print(sample["sentence"])
    # count += 1
    # if count == 10:
    #     break

Classifing: 100%|██████████| 10000/10000 [06:58<00:00, 23.88it/s]


In [47]:
acc_computer.compute()

{'accuracy': 0.0803}

In [48]:
print(f"{0.0803 * 100}%")

8.03%
